#The Modern Mamak: Exploring the New Socio-spatial Roles of Mamak Shops in Modern Singapore

“Mamak” stalls, traditionally small provision shops established by Indian and Sri Lankan migrants, have long been essential fixtures in Singapore’s public housing estates. Historically, these stalls sold groceries and sundry items, serving the everyday needs in local communities. These stores also serve certain cultural demands for traditional spices and ingredients that often originate from India or China. The term "mamak" comes from the Tamil word "maa-ma," meaning “uncle”, as these were often honorifics used by children in Singapore to respectfully address adults in these stores. At their peak in the early 2000s, there were around 3,000 mamak stalls; however, due to increased competition from chain convenience stores, large supermarkets, and online shopping, the number has dwindled to just about a few hundred today.

The decline of mamak stalls signified an economic and cultural shift. As housing developments experience intensive urban development and the advent of growing corporations, traditional mamak stalls are being outcompeted by larger, corporatized grocery chains and convenient stores that lack the unique character these shops once brought to public housing estates. Over the years, this change has made the experience of living in public housing less distinctive and more homogenised. Moreover, this shift risks outpricing lower income demographics of residents, catering primarily to residents with greater financial ability. This represents a form of spatial injustice within local communities. Additionally, these mamak stalls were also the primary income for small business owners, some of whom now try to adopt new roles as collection points for e-commerce delivery. Other mamak stores have since closed down as their traditional businesses models have become less viable.

This project utilises a couple of data sources provided by Singapore's state-owned organisations. The shop dataset in this document is collected from the Accounting and Corporate Regulatory Authority of Singapore (ACRA) with the following as the definition of mamak shops:

* Shops with Primary SSIC code 47102 (mini-marts, convenience stores and provision shops)
* Shops without a Secondary SSIC (shops without secondary functions)
* Shops that are 'Sole Proprietory' businesses

<br>

###Contents:
1. Create New Dataframe of Only Relevant Columns
2. Filter By Primary SSIC Code 47102
3. Filter Out Shops with Secondary Functions
4. Filter Out non Sole-Proprietory Businesses
5. Data Cleaning and Sorting into Decades
6. Summary of Preliminary Analysis

# Libraries

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Google credentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Files

In [3]:
file_path = '/content/drive/MyDrive/Mamak Stores/ACRA data/all_acra_businesses.csv'
comb_df = pd.read_csv(file_path)

<ipython-input-3-b66bb05416f5>:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  comb_df = pd.read_csv(file_path)


In [4]:
comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982005 entries, 0 to 1982004
Data columns (total 53 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   uen                                object
 1   issuance_agency_id                 object
 2   entity_name                        object
 3   entity_type_description            object
 4   business_constitution_description  object
 5   company_type_description           object
 6   paf_constitution_description       object
 7   entity_status_description          object
 8   registration_incorporation_date    object
 9   uen_issue_date                     object
 10  address_type                       object
 11  block                              object
 12  street_name                        object
 13  level_no                           object
 14  unit_no                            object
 15  building_name                      object
 16  postal_code                        o

#1. New Dataframe of Only Relevant Columns

Only columns 0 - 26 will be used and hence the rest will be filtered out for easier reading and comprehension. The other columns that are pertaining to audit details are not relevant to the project.

In [5]:
col_list = list(comb_df.columns[:27])
print(col_list)

['uen', 'issuance_agency_id', 'entity_name', 'entity_type_description', 'business_constitution_description', 'company_type_description', 'paf_constitution_description', 'entity_status_description', 'registration_incorporation_date', 'uen_issue_date', 'address_type', 'block', 'street_name', 'level_no', 'unit_no', 'building_name', 'postal_code', 'other_address_line1', 'other_address_line2', 'account_due_date', 'annual_return_date', 'primary_ssic_code', 'primary_ssic_description', 'primary_user_described_activity', 'secondary_ssic_code', 'secondary_ssic_description', 'secondary_user_described_activity']


In [6]:
comb_df_2 = comb_df[col_list]

#2. Filter by Primary SSIC Code 47102

The SSIC is the Singapore Standard of Industrial Classification used to categorise business entities in the country. 47102 refers to mini-marts, convenience stores and provision shops. Many traditional mamak shops were classified under this category dating from 1904 until today. However, this includes other larger chain stores as well. To combat this, other filters will be implemented to remove irrelevant entities from the dataset. Records before the 1980s from ACRA may also not be as reliable, hence the dataset of mamak shops will be constrained within the the timeframe from 1980 until now.

In [7]:
comb_df_2['primary_ssic_code'].astype(int)

,primary_ssic_code
0,58110
1,24201
2,70102
3,56111
4,42909
...,...
1982000,47213
1982001,66303
1982002,64202
1982003,64202


In [8]:
mamaks_df = comb_df_2[comb_df_2['primary_ssic_code'] == 47102]
mamaks_df.reset_index(inplace = True, drop = True)
mamaks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14454 entries, 0 to 14453
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   uen                                14454 non-null  object
 1   issuance_agency_id                 14454 non-null  object
 2   entity_name                        14453 non-null  object
 3   entity_type_description            14454 non-null  object
 4   business_constitution_description  14454 non-null  object
 5   company_type_description           14454 non-null  object
 6   paf_constitution_description       14454 non-null  object
 7   entity_status_description          14454 non-null  object
 8   registration_incorporation_date    14454 non-null  object
 9   uen_issue_date                     14454 non-null  object
 10  address_type                       14454 non-null  object
 11  block                              14454 non-null  object
 12  stre

#3. Filter Out Shops with Secondary Functions

Many provision shops also have secondary roles, such as pawn brokers and currency exchangers. These shops are less vulnerable to closure and do not fall within the traditional definitions of provision shops. Including these shops in the dataset may skew the results, therefore they will be filtered out.

In [9]:
mamaks_df['secondary_ssic_code'] = mamaks_df['secondary_ssic_code'].replace('na', np.nan)

<ipython-input-9-6583cbb1c83e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mamaks_df['secondary_ssic_code'] = mamaks_df['secondary_ssic_code'].replace('na', np.nan)


In [10]:
mamaks_filtered = mamaks_df[mamaks_df['secondary_ssic_code'].isnull()]

mamaks_filtered.reset_index(inplace = True, drop = True)
mamaks_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8060 entries, 0 to 8059
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   uen                                8060 non-null   object
 1   issuance_agency_id                 8060 non-null   object
 2   entity_name                        8059 non-null   object
 3   entity_type_description            8060 non-null   object
 4   business_constitution_description  8060 non-null   object
 5   company_type_description           8060 non-null   object
 6   paf_constitution_description       8060 non-null   object
 7   entity_status_description          8060 non-null   object
 8   registration_incorporation_date    8060 non-null   object
 9   uen_issue_date                     8060 non-null   object
 10  address_type                       8060 non-null   object
 11  block                              8060 non-null   object
 12  street

#4. Filter Out Shops that are not Sole Prioprietory or Partnership Businesses

Most mamak shops are Sole Proprietorship businesses and not LLPs or Limited Liability Partnerships. The shopowner often run these stores by themselves or with the assistance of family members. They are also personally liable for all business debts and obligations, which puts them at a disadvantage compared to convenience store franchises and corporatised supermarkets.

In [11]:
mamaks_filtered = mamaks_filtered[
    mamaks_filtered['business_constitution_description'].isin(['Partnership', 'Sole Proprietor'])
].reset_index(drop=True)

mamaks_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7438 entries, 0 to 7437
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   uen                                7438 non-null   object
 1   issuance_agency_id                 7438 non-null   object
 2   entity_name                        7437 non-null   object
 3   entity_type_description            7438 non-null   object
 4   business_constitution_description  7438 non-null   object
 5   company_type_description           7438 non-null   object
 6   paf_constitution_description       7438 non-null   object
 7   entity_status_description          7438 non-null   object
 8   registration_incorporation_date    7438 non-null   object
 9   uen_issue_date                     7438 non-null   object
 10  address_type                       7438 non-null   object
 11  block                              7438 non-null   object
 12  street

In [12]:
mamaks_filtered.tail()

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,other_address_line1,other_address_line2,account_due_date,annual_return_date,primary_ssic_code,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity
7433,53437288X,ACRA,EMINENT 24-7 CONVENIENCE,Business,Sole Proprietor,na,na,Cancelled (Non-Renewal),2021-07-26,2021-07-26,...,na,na,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...","SALE OF DRINKS, SNACKS, GROCERY, ETC",NaN,na,"SALE OF DRINKS, SNACKS, GROCERY, ETC"
7434,53437549M,ACRA,E-ZY CORNER,Business,Sole Proprietor,na,na,Cancelled (Non-Renewal),2021-07-31,2021-07-31,...,na,na,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...","MINI-MARTS,CONVENIENCE STORES AND PROVISION SHOP",NaN,na,"MINI-MARTS,CONVENIENCE STORES AND PROVISION SHOP"
7435,53441172W,ACRA,ELPIS ENTERPRISE,Business,Sole Proprietor,na,na,Live,2021-10-16,2021-10-16,...,na,na,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",47112,NaN,na,47112
7436,53472771M,ACRA,ED@KAI,Business,Partnership,na,na,Live,2023-08-28,2023-08-28,...,na,na,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",47102,NaN,na,47102
7437,53484972E,ACRA,EVERMART,Business,Sole Proprietor,na,na,Live,2024-05-02,2024-05-02,...,na,na,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na


In [13]:
def remove_corp(df):
    corp_list = ['7-ELEVEN', 'CHEERS', 'ECON', 'FRANCHISEE', '7-11']
    pattern = '|'.join(corp_list)

    df = df[~df['entity_name'].str.contains(pattern, case=False, na=False)].copy()
    df = df[~df['primary_user_described_activity'].str.contains(pattern, case=False, na=False)].copy()

    return df

In [14]:
mamaks_filtered = remove_corp(mamaks_filtered)

In [15]:
mamaks_filtered['registration_incorporation_date'] = pd.to_datetime(
    mamaks_filtered['registration_incorporation_date'], errors='coerce')

mamaks_filtered['registration_year'] = mamaks_filtered['registration_incorporation_date'].dt.year

In [16]:
mamaks_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7392 entries, 0 to 7437
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   uen                                7392 non-null   object        
 1   issuance_agency_id                 7392 non-null   object        
 2   entity_name                        7391 non-null   object        
 3   entity_type_description            7392 non-null   object        
 4   business_constitution_description  7392 non-null   object        
 5   company_type_description           7392 non-null   object        
 6   paf_constitution_description       7392 non-null   object        
 7   entity_status_description          7392 non-null   object        
 8   registration_incorporation_date    7392 non-null   datetime64[ns]
 9   uen_issue_date                     7392 non-null   object        
 10  address_type                       7392 n

#5. Data Cleaning and Sorting into Decades

### Combine Address Information into One Column ('Address')

Following this data cleaning process, the addresses of these entities will be used for geocoding using Google API.

In [17]:
cols_1 = ['block', 'street_name']
cols_2 = ['building_name']

#address_1 (block + street_name)
address_1 = (
    mamaks_filtered[cols_1]
    .fillna('')
    .astype(str)
    .replace('na', '')
    .agg(' '.join, axis=1)
    .str.strip()
)

#'#level_no-unit_no' (if both exist)
level_unit = mamaks_filtered[['level_no', 'unit_no']].fillna('').astype(str).replace('na', '')
address_2 = level_unit.apply(
    lambda x: f"#{x.level_no}-{x.unit_no}" if x.level_no and x.unit_no
    else f"#{x.unit_no}" if x.unit_no
    else '',
    axis=1
)

#combine with building_name
address_2 += ' ' + (
    mamaks_filtered[cols_2]
    .fillna('')
    .astype(str)
    .replace('na', '')
    .agg(' '.join, axis=1)
    .str.strip()
)

#add 'SINGAPORE' before postal code
address_3 = mamaks_filtered['postal_code'].fillna('').astype(str).replace('na', '').apply(
    lambda x: f"SINGAPORE {x}"
)

#full address
mamaks_filtered['Address'] = (
    address_1 + ' ' + address_2.str.strip() + ' ' + address_3
).str.replace(r'\s+', ' ', regex=True).str.strip()


In [18]:
mamaks_filtered.tail(30)

,uen,issuance_agency_id,entity_name,entity_type_description,business_constitution_description,company_type_description,paf_constitution_description,entity_status_description,registration_incorporation_date,uen_issue_date,...,account_due_date,annual_return_date,primary_ssic_code,primary_ssic_description,primary_user_described_activity,secondary_ssic_code,secondary_ssic_description,secondary_user_described_activity,registration_year,Address
7406,53146408C,ACRA,EC-AC MARK,Business,Sole Proprietor,na,na,Live,2009-07-03,2009-07-04,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2009,25 AIRLINE ROAD AIRLINE HOUSE SINGAPORE 819829
7407,53156223M,ACRA,EXPRESS MART @ MARINA,Business,Sole Proprietor,na,na,Cancelled,2009-11-19,2009-11-20,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2009,6 RAFFLES BOULEVARD #02-257 MARINA SQUARE SING...
7408,53178268M,ACRA,EXPRESS AVENUE,Business,Partnership,na,na,Live,2010-10-21,2010-10-22,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2010,58 MARINE TERRACE #01-43 MARINE TERRACE HAVEN ...
7409,53188213J,ACRA,ERP MINIMART,Business,Sole Proprietor,na,na,Cancelled (Non-Renewal),2011-03-26,2011-03-27,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",CONVENIENCE STORE,NaN,na,CONVENIENCE STORE,2011,23B QUEEN'S CLOSE #01-183 SINGAPORE 141023
7410,53211441L,ACRA,EXPRESS MART @ CLARKE QUAY,Business,Sole Proprietor,na,na,Cancelled,2012-03-23,2012-03-24,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",MINI-MART,NaN,na,MINI-MART,2012,1 ESPLANADE DRIVE #01-08A ESPLANADE - THEATRES...
7411,53211974J,ACRA,ECW TREASURE POTS,Business,Sole Proprietor,na,na,Cancelled,2012-03-30,2012-03-31,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2012,291D COMPASSVALE STREET #08-276 SINGAPORE 544291
7412,53222955W,ACRA,E-MART_,Business,Sole Proprietor,na,na,Cancelled (Non-Renewal),2012-09-20,2012-09-21,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",GENERAL GROCERY SUPPLIER FRUITS AND VEG,NaN,na,GENERAL GROCERY SUPPLIER FRUITS AND VEG,2012,34 DOVER ROAD #01-143 SINGAPORE 130034
7413,53223118W,ACRA,ERVADI MINI MART,Business,Sole Proprietor,na,na,na,2012-09-22,2012-09-23,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2012,54 CASHEW ROAD #02-03 CASHEW PARK SINGAPORE 67...
7414,53225747B,ACRA,EJAAN,Business,Sole Proprietor,na,na,Ceased Registration,2012-11-05,2012-11-06,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2012,258B PUNGGOL FIELD #6-07 PUNGGOL TOPAZ SINGAPO...
7415,53232138C,ACRA,EMMANUEL MINIMART,Business,Sole Proprietor,na,na,Live,2013-03-03,2013-03-04,...,na,na,47102,"MINI-MARTS, CONVENIENCE STORES AND PROVISION S...",na,NaN,na,na,2013,819 TAMPINES STREET 81 #01-K1 TAMPINES POLYVIE...


### Get All Currently Live Provision Shops

In [19]:
mamaks_now_live = mamaks_filtered.copy()
mamaks_now_live = mamaks_now_live[
    (mamaks_now_live['entity_status_description'] == 'Live') |
    (mamaks_now_live['entity_status_description'] == 'Live Company')
]

mamaks_now_live.reset_index(inplace = True, drop = True)

mamaks_now_live.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   uen                                656 non-null    object        
 1   issuance_agency_id                 656 non-null    object        
 2   entity_name                        656 non-null    object        
 3   entity_type_description            656 non-null    object        
 4   business_constitution_description  656 non-null    object        
 5   company_type_description           656 non-null    object        
 6   paf_constitution_description       656 non-null    object        
 7   entity_status_description          656 non-null    object        
 8   registration_incorporation_date    656 non-null    datetime64[ns]
 9   uen_issue_date                     656 non-null    object        
 10  address_type                       656

In [20]:
#SAVE
mamaks_filtered.to_csv('/content/drive/MyDrive/Mamak Stores/ACRA data/mm_all.csv', index=False)
mamaks_now_live.to_csv('/content/drive/MyDrive/Mamak Stores/ACRA data/mm_2025_live.csv', index=False)